# Example
This notebook illustrates how the sleepwellbaby package can be used to get sleep state predictions using mock data.

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

from sleepwellbaby.data import (
    compute_reference_values,
    generate_mock_signalbase_data,
)
from sleepwellbaby.explainability import dot_plot, get_shap_values
from sleepwellbaby.model import load_model
from sleepwellbaby.utils import get_swb_predictions



## Prepare data
Generate some mock data (same format as SignalBase with 1Hz). If you want to use 0.4Hz set `freq=2s500ms`.

In [ ]:
df = generate_mock_signalbase_data(freq='S').sort_values(by='datetime').set_index('datetime')

Compute reference values using rolling windows

In [ ]:
df = compute_reference_values(df, freq=1)

## Get predictions 
We make a prediction every one minute in the below example.

*Note*: we start 8 minutes later than our dataframe otherwise we get a ValueError in `get_swb_predictions`.

In [ ]:
# Get indices of times at which you want to get an SWB value (we only compute it at whole minutes)
t_range_swb = pd.date_range(
    start=(df.index.min().round(freq='min') + pd.Timedelta(minutes=8)),
    end=df.index.max().round(freq='min'),
    freq='1min'
)

Return predictions (takes a couple of minutes for a single day with one prediction every minute, tsfresh limits the speed).

In [ ]:
df, columns = get_swb_predictions(df, t_range_swb[-50:], birth_date="2000-01-01", gestation_period=210, freq='S', return_features=True)

Drop rows without a prediction

In [ ]:
df.dropna(subset='prediction', inplace=True)

In [ ]:
df.tail(10)

### Get shap values

In [ ]:
mask = df.loc[:,'prediction']!='ineligible'
model, model_support_dict = load_model()
X = df.loc[mask, model_support_dict['Xcol']].reset_index(drop=True)

assert X.isna().any().sum() == 0
assert (X.dtypes == np.float64).all()  # otherwise if, e.g. object, get all grey valuess

shap_values = get_shap_values(model, X)

In [ ]:
figs = dot_plot(shap_values, X, model.classes_)

In [ ]:
figs[0]

In [ ]:
figs[1]

In [ ]:
figs[2]

#